<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#tf.map_fn" data-toc-modified-id="tf.map_fn-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>tf.map_fn</a></span></li></ul></div>

In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
%watermark -p torch,torchvision,torchaudio
%watermark -p tensorflow,tensorboard,tflite
%watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Image, Javascript
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, time, random, math
import json, base64, requests, shutil
import os.path as osp
import numpy as np
import argparse, shlex, signal

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x):
    try:
        x = x.strip()
        if x.startswith('https://'):
            x = x[8:]
        if not x.endswith('.py'):
            x = x + '.py'
        if x[0] == '/':
            with open(x) as fr:
                x = fr.read()
        else:
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                uri = 'https://' + x
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                uri = x.replace('github.com', 'raw.githubusercontent.com')
                mod = uri.split('/')
                for s in ['main', 'master']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except:
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)


numpy 1.19.5
sklearn 0.0
pandas 1.1.5
ipywidgets 7.6.3
cv2 4.5.3
PIL 8.3.1
matplotlib 3.3.4
plotly 5.3.0
netron 5.1.6
torch 1.8.1+cu101
torchvision 0.9.1+cu101
torchaudio not installed
tensorflow 2.6.0
tensorboard 2.6.0
tflite not installed
onnx 1.10.1
tf2onnx 1.10.0
onnxruntime 1.8.1
tensorrt not installed
tvm not installed


In [2]:
def display_html(port, height=600):
    from IPython import display
    from html import escape as html_escape
    frame_id = 'erlangai-frame-{:08x}'.format(random.getrandbits(64))
    shell = '''
      <iframe id='%HTML_ID%' width='100%' height='%HEIGHT%' frameborder='0'>
      </iframe>
      <script>
        (function() {
          const frame = document.getElementById(%JSON_ID%);
          const url = new URL(%URL%, window.location);
          const port = %PORT%;
          if (port) {
            url.port = port;
          }
          frame.src = url;
        })();
      </script>
    '''
    replacements = [
        ('%HTML_ID%', html_escape(frame_id, quote=True)),
        ('%JSON_ID%', json.dumps(frame_id)),
        ('%HEIGHT%', '%d' % height),
        ('%PORT%', '%d' % port),
        ('%URL%', json.dumps('/')),
    ]
    for (k, v) in replacements:
        shell = shell.replace(k, v)
    display.display(display.HTML(shell))
    
@register_line_magic
def netron(line):
    parser = argparse.ArgumentParser(prog='netron')
    parser.add_argument('--file', '-f', type=str, required=True, help='netron model file')
    parser.add_argument('--port', '-p', type=int, default=0, help='netron server port')
    parser.add_argument('--height', type=int, default=500, help='display netron html window hight')
    import netron
    try:
        args = parser.parse_args(shlex.split(line))
        address = netron.start(args.file, address=('0.0.0.0', args.port), browse=False)
        display_html(address[1], args.height)
    except:
        pass


In [3]:

###
### Tensorflow ###
###

import tensorflow as tf
import tf2onnx


NB_TMP_DATA='/data/nb_data/tmp'

## tf.map_fn

In [4]:
@tf.function
def test_f1(x):
    return tf.map_fn(lambda y: y + 2, x)

@tf.function
def test_f2(x):
    # a = tf.expand_dims(x[0]+2, 0)
    # b = tf.expand_dims(x[1]+2, 0)
    # c = tf.expand_dims(x[2]+2, 0)
    # return tf.concat([a, b, c], 0)
    z = tf.TensorArray(tf.float32, size=x.shape[0])
    for i in tf.range(x.shape[0]):
        z = z.write(i, x[i] + 2)
    return z.stack()
    
func_proto = tf2onnx.convert.from_function(
    function=test_f2, opset=11,
    input_signature=(
        tf.TensorSpec(shape=(3, 2, 2), name="x"),
    ),
    output_path=f'{NB_TMP_DATA}/tf_array.onnx'
)[0];

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


ONNX Failed to infer shapes and dtypes for [while/strided_slice__34, type: Squeeze]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tf2onnx/schemas.py", line 154, in infer_onnx_shape_dtype
    inferred_model = shape_inference.infer_shapes(model_proto, strict_mode=True)
  File "/usr/local/lib/python3.6/dist-packages/onnx/shape_inference.py", line 42, in infer_shapes
    inferred_model_str = C.infer_shapes(model_str, check_type, strict_mode, data_prop)
onnx.onnx_cpp2py_export.shape_inference.InferenceError: [ShapeInferenceError] Shape inference error(s): (op_type:Squeeze, node name: while/strided_slice__34): [ShapeInferenceError] Dimension of input 0 must be 1 instead of 2



In [5]:
x = [
    [[1, 10], [2, 20]],
    [[3, 30], [4, 40]],
    [[5, 50], [6, 60]],
]

x = np.array(x, dtype=np.float32)
x.shape, x, '-'*60, test_f1(x), '-'*60, test_f2(x)

((3, 2, 2),
 array([[[ 1., 10.],
         [ 2., 20.]],
 
        [[ 3., 30.],
         [ 4., 40.]],
 
        [[ 5., 50.],
         [ 6., 60.]]], dtype=float32),
 '------------------------------------------------------------',
 <tf.Tensor: shape=(3, 2, 2), dtype=float32, numpy=
 array([[[ 3., 12.],
         [ 4., 22.]],
 
        [[ 5., 32.],
         [ 6., 42.]],
 
        [[ 7., 52.],
         [ 8., 62.]]], dtype=float32)>,
 '------------------------------------------------------------',
 <tf.Tensor: shape=(3, 2, 2), dtype=float32, numpy=
 array([[[ 3., 12.],
         [ 4., 22.]],
 
        [[ 5., 32.],
         [ 6., 42.]],
 
        [[ 7., 52.],
         [ 8., 62.]]], dtype=float32)>)

In [6]:
func_proto = tf2onnx.convert.from_function(
    function=test_f1, opset=11,
    input_signature=(
        tf.TensorSpec(shape=(None, 2, 2), name="x"),
    ),
    output_path=f'{NB_TMP_DATA}/map_fn.onnx'
)[0];

In [7]:
# from tf2onnx import logging, utils
# logging.basicConfig(level=0)
# utils.set_debug_mode(False)
func_proto = tf2onnx.convert.from_function(
    function=test_f2, opset=11,
    input_signature=(
        tf.TensorSpec(shape=(3, 2, 2), name="x"),
    ),
    output_path=f'{NB_TMP_DATA}/tf_array.onnx'
)[0];

ONNX Failed to infer shapes and dtypes for [while/strided_slice__99, type: Squeeze]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tf2onnx/schemas.py", line 154, in infer_onnx_shape_dtype
    inferred_model = shape_inference.infer_shapes(model_proto, strict_mode=True)
  File "/usr/local/lib/python3.6/dist-packages/onnx/shape_inference.py", line 42, in infer_shapes
    inferred_model_str = C.infer_shapes(model_str, check_type, strict_mode, data_prop)
onnx.onnx_cpp2py_export.shape_inference.InferenceError: [ShapeInferenceError] Shape inference error(s): (op_type:Squeeze, node name: while/strided_slice__99): [ShapeInferenceError] Dimension of input 0 must be 1 instead of 2

